# Testing model building with `carveme` and `memote`

We firstly tried: Importing the COBRA module to read sbml models created via the `carveme` method. Subsequently we reconstruct a draft of the GSM model of *S. cerevisae* using the found RefSeq ID. Looking at the [reference genome R64](https://www.ncbi.nlm.nih.gov/data-hub/taxonomy/4932/?utm_source=assembly&utm_medium=referral&utm_campaign=KnownItemSensor:taxname), we see that 6464 have been annotated. 

But found a model online that worked better

In [1]:
from cobra.io import read_sbml_model
from cobra import Reaction, Metabolite
import escher

%%capture
!carve --refseq GCF_000146045.2_R64 --output Saccharomyces.xml

print("DONE WITH THE CARVING!")

In [2]:
model = read_sbml_model('../yeast-GEM.xml') # Model initiated in minimal medium + glucose (flux bound = 1.0)

In [3]:
print("GSM model fact sheet:")
print("Number of reactions:", len(model.reactions))
print("Number of metabolites:", len(model.metabolites))
print("Number of genes included in the model:", len(model.genes))
print("% of genes in model compared to the number of coding genes in Saccharomyces cerevisae R64 (6464 genes):",
      round((len(model.genes)/6464)*100, 2))
print("Gene coverage percentage =", round(len(model.genes)/len(model.reactions)*100, 2), "%")

GSM model fact sheet:
Number of reactions: 4058
Number of metabolites: 2742
Number of genes included in the model: 1150
% of genes in model compared to the number of coding genes in Saccharomyces cerevisae R64 (6464 genes): 17.79
Gene coverage percentage = 28.34 %


Checking the model setup - how the metabolites etc. are named/tagged

In [4]:
model.metabolites[:5]

[<Metabolite s_0001[ce] at 0x7f03d8da7d30>,
 <Metabolite s_0002[c] at 0x7f03d8da7e50>,
 <Metabolite s_0003[e] at 0x7f03d8da7fa0>,
 <Metabolite s_0004[ce] at 0x7f03d8da7f40>,
 <Metabolite s_0006[m] at 0x7f03d8da7ee0>]

In [5]:
model.compartments # compartments used

{'ce': 'cell envelope',
 'c': 'cytoplasm',
 'e': 'extracellular',
 'm': 'mitochondrion',
 'n': 'nucleus',
 'p': 'peroxisome',
 'er': 'endoplasmic reticulum',
 'g': 'Golgi',
 'lp': 'lipid particle',
 'v': 'vacuole',
 'erm': 'endoplasmic reticulum membrane',
 'vm': 'vacuolar membrane',
 'gm': 'Golgi membrane',
 'mm': 'mitochondrial membrane'}

In [6]:
model.metabolites.get_by_id('s_0001[ce]') # Again specifying model naming

Metabolite identifier,s_0001[ce]
Name,(1->3)-beta-D-glucan [cell envelope]
Memory address,0x07f03d8da7d30
Formula,C6H10O5
Compartment,ce
In 3 reaction(s),"r_1543, r_0005, r_4048"


In [7]:
# Investigating where in the model, acetyl-CoA is used
for metabolite in model.metabolites.query('acetyl-CoA', 'name'):
    print(metabolite.name)

acetoacetyl-CoA [cytoplasm]
acetoacetyl-CoA [mitochondrion]
acetyl-CoA [cytoplasm]
acetyl-CoA [mitochondrion]
acetyl-CoA [nucleus]
acetyl-CoA [peroxisome]
acetoacetyl-CoA [peroxisome]


In [8]:
# Model default objective function (growth rate)
print("Model objective function:\n", model.objective, sep="")

Model objective function:
Maximize
1.0*r_2111 - 1.0*r_2111_reverse_58b69


In [9]:
# Investigating the growth reaction rate
model.reactions.get_by_id('r_2111')

Reaction identifier,r_2111
Name,growth
Memory address,0x07f03d8c7d880
Stoichiometry,s_0450[c] --> biomass [cytoplasm] -->
GPR,
Lower bound,0.0
Upper bound,1000.0


In [10]:
model.metabolites.get_by_id('s_0450[c]')

Metabolite identifier,s_0450[c]
Name,biomass [cytoplasm]
Memory address,0x07f03d8d17f40
Formula,
Compartment,c
In 2 reaction(s),"r_2111, r_4041"


In [11]:
model.optimize()

,fluxes,reduced_costs
r_0001,0.000000,-1.667286e-02
r_0002,0.000000,-1.667286e-02
r_0003,0.000000,-0.000000e+00
r_0004,0.000000,1.734723e-18
r_0005,0.062686,6.722053e-18
...,...,...
r_4694,0.000000,0.000000e+00
r_4695,0.000000,0.000000e+00
r_4697,0.000000,-0.000000e+00
r_4698,0.000000,-1.866638e-01


In [12]:
# Exchange reactions for uptake of nutrients from medium
for r in model.medium.keys():
    print(r, "|", model.reactions.get_by_id(r).name, "=",model.medium[r])

r_1654 | ammonium exchange = 1000.0
r_1714 | D-glucose exchange = 1.0
r_1832 | H+ exchange = 1000.0
r_1861 | iron(2+) exchange = 1000.0
r_1992 | oxygen exchange = 1000.0
r_2005 | phosphate exchange = 1000.0
r_2020 | potassium exchange = 1000.0
r_2049 | sodium exchange = 1000.0
r_2060 | sulphate exchange = 1000.0
r_2100 | water exchange = 1000.0
r_4593 | chloride exchange = 1000.0
r_4594 | Cu2(+) exchange = 1000.0
r_4595 | Mn(2+) exchange = 1000.0
r_4596 | Zn(2+) exchange = 1000.0
r_4597 | Mg(2+) exchange = 1000.0
r_4600 | Ca(2+) exchange = 1000.0


In [13]:
# Available exchange reactions for our model
for reaction in model.exchanges:
    print(reaction.id, "|", reaction.name)

r_1542 | (1->3)-beta-D-glucan exchange
r_1545 | (R)-carnitine exchange
r_1546 | (R)-lactate exchange
r_1547 | (R)-mevalonate exchange
r_1548 | (R)-pantothenate exchange
r_1549 | (R,R)-2,3-butanediol exchange
r_1550 | (S)-3-methyl-2-oxopentanoate exchange
r_1551 | (S)-lactate exchange
r_1552 | (S)-malate exchange
r_1553 | 1-(sn-glycero-3-phospho)-1D-myo-inositol exchange
r_1554 | 1-acylglycerophosphocholine exchange
r_1563 | 2'-deoxyadenosine exchange
r_1564 | 2'-deoxyguanosine exchange
r_1565 | 2'-deoxyinosine exchange
r_1566 | 2'-deoxyuridine exchange
r_1572 | 2-isopropylmalate exchange
r_1577 | 2-methylbutanal exchange
r_1580 | 2-methylbutanol exchange
r_1581 | 2-methylbutyl acetate exchange
r_1586 | 2-oxoglutarate exchange
r_1589 | 2-phenylethanol exchange
r_1598 | 3-methylbutanal exchange
r_1604 | 4-aminobenzoate exchange
r_1621 | 5-aminolevulinate exchange
r_1625 | 5-formyltetrahydrofolic acid exchange
r_1627 | 7,8-diaminononanoate exchange
r_1629 | 8-amino-7-oxononanoate exchange

In [14]:
model.reactions.r_1714

Reaction identifier,r_1714
Name,D-glucose exchange
Memory address,0x07f03efe72310
Stoichiometry,s_0565[e] <=> D-glucose [extracellular] <=>
GPR,
Lower bound,-1.0
Upper bound,1000.0


In [15]:
model.metabolites.get_by_id('s_0565[e]')

Metabolite identifier,s_0565[e]
Name,D-glucose [extracellular]
Memory address,0x07f03f156f1f0
Formula,C6H12O6
Compartment,e
In 6 reaction(s),"r_1166, r_1714, r_0370, r_1024, r_4420, r_4400"


In [16]:
# Experimentation with glucose concentration --> reach maximum growth rate at conc. ca. 575 
medium = model.medium
min_gluc = model.optimize()
print("mu_min_gluc =", min_gluc.fluxes['r_2111'])
with model:
    medium['r_1714'] = 575  # Glucose exchange reaciton
    model.medium = medium
    solution = model.optimize()
    print("mu_max_gluc =",solution.fluxes['r_2111'])

mu_min_gluc = 0.08374778664999834
mu_max_gluc = 19.816830259464282


In [17]:
for reaction in model.metabolites.get_by_id('s_0367[c]').reactions:
    reaction
    print(reaction.name, reaction, sep = " | ")

acetyl-CoA C-acetyltransferase | r_0103: 2.0 s_0373[c] <=> s_0367[c] + s_0529[c]
Acetoacetate:CoA ligase (AMP-forming) | r_4396: s_0434[c] + s_0529[c] + s_4132[c] --> s_0367[c] + s_0423[c] + s_0633[c]
hydroxymethylglutaryl CoA synthase | r_0559: s_0367[c] + s_0373[c] + s_0803[c] --> s_0218[c] + s_0529[c] + s_0794[c]


In [18]:
model.metabolites.get_by_id('s_0367[c]').reactions

frozenset({<Reaction r_0103 at 0x7f03f0f53520>,
           <Reaction r_0559 at 0x7f03f0c99c70>,
           <Reaction r_4396 at 0x7f03d8893c70>})

In [19]:
for r in model.reactions.query('maintenance', 'name'):
    print(r.id, r.name)

r_4046 non-growth associated maintenance reaction


In [28]:
model.reactions.r_4046

Reaction identifier,r_4046
Name,non-growth associated maintenance reaction
Memory address,0x07f03d8ac4f70
Stoichiometry,s_0434[c] + s_0803[c] --> s_0394[c] + s_0794[c] + s_1322[c] ATP [cytoplasm] + H2O [cytoplasm] --> ADP [cytoplasm] + H+ [cytoplasm] + phosphate [cytoplasm]
GPR,
Lower bound,0.7
Upper bound,0.7


# code for changing objective function
with model:
    model.objective = model.reactions.'reaction # <-- here we insert the reaction we want as objective
    print(model.optimize().objective_value)

In [94]:
for m in model.metabolites.query('geranyl', 'name'):
    print(m.id, m.name, model.metabolites.get_by_id(str(m.id)).reactions, sep='\n')

s_0189[c]
geranylgeranyl diphosphate [cytoplasm]
frozenset({<Reaction r_0461 at 0x7f03f0da4e80>, <Reaction r_0373 at 0x7f03f0da48e0>})
s_0745[c]
geranyl diphosphate [cytoplasm]
frozenset({<Reaction r_0462 at 0x7f03f0d64eb0>, <Reaction r_0355 at 0x7f03f0d90fd0>})
s_0746[lp]
geranylgeranyl diphosphate [lipid particle]
frozenset({<Reaction r_0282 at 0x7f03f0e83dc0>, <Reaction r_0281 at 0x7f03f0e72be0>})


In [81]:
for r in model.metabolites.get_by_id('s_0218[c]').reactions:
    print(r.name, '\n', r)

hydroxymethylglutaryl CoA reductase 
 r_0558: s_0218[c] + 2.0 s_0794[c] + 2.0 s_1212[c] --> s_0028[c] + s_0529[c] + 2.0 s_1207[c]
hydroxymethylglutaryl-CoA transport 
 r_1840: s_0218[c] <=> s_0221[m]
hydroxymethylglutaryl CoA synthase 
 r_0559: s_0367[c] + s_0373[c] + s_0803[c] --> s_0218[c] + s_0529[c] + s_0794[c]


In [85]:
for boundary in model.boundary:
    print(boundary.name)

(1->3)-beta-D-glucan exchange
(R)-carnitine exchange
(R)-lactate exchange
(R)-mevalonate exchange
(R)-pantothenate exchange
(R,R)-2,3-butanediol exchange
(S)-3-methyl-2-oxopentanoate exchange
(S)-lactate exchange
(S)-malate exchange
1-(sn-glycero-3-phospho)-1D-myo-inositol exchange
1-acylglycerophosphocholine exchange
2'-deoxyadenosine exchange
2'-deoxyguanosine exchange
2'-deoxyinosine exchange
2'-deoxyuridine exchange
2-isopropylmalate exchange
2-methylbutanal exchange
2-methylbutanol exchange
2-methylbutyl acetate exchange
2-oxoglutarate exchange
2-phenylethanol exchange
3-methylbutanal exchange
4-aminobenzoate exchange
5-aminolevulinate exchange
5-formyltetrahydrofolic acid exchange
7,8-diaminononanoate exchange
8-amino-7-oxononanoate exchange
9H-xanthine exchange
acetaldehyde exchange
acetate exchange
adenine exchange
adenosine 3',5'-bismonophosphate exchange
adenosine exchange
allantoate exchange
allantoin exchange
trehalose exchange
alpha-D-glucosamine 6-phosphate exchange
ammon

In [93]:
model.exchanges

[<Reaction r_1542 at 0x7f03efee0700>,
 <Reaction r_1545 at 0x7f03efee0a90>,
 <Reaction r_1546 at 0x7f03efee0ca0>,
 <Reaction r_1547 at 0x7f03efee0f10>,
 <Reaction r_1548 at 0x7f03efee0e50>,
 <Reaction r_1549 at 0x7f03efef83d0>,
 <Reaction r_1550 at 0x7f03efef8550>,
 <Reaction r_1551 at 0x7f03efef8580>,
 <Reaction r_1552 at 0x7f03efef8640>,
 <Reaction r_1553 at 0x7f03efef8730>,
 <Reaction r_1554 at 0x7f03efef8790>,
 <Reaction r_1563 at 0x7f03efef8910>,
 <Reaction r_1564 at 0x7f03efef89a0>,
 <Reaction r_1565 at 0x7f03efef8a00>,
 <Reaction r_1566 at 0x7f03efef8a30>,
 <Reaction r_1572 at 0x7f03efef8bb0>,
 <Reaction r_1577 at 0x7f03effc2fd0>,
 <Reaction r_1580 at 0x7f03efef8d60>,
 <Reaction r_1581 at 0x7f03efef8e80>,
 <Reaction r_1586 at 0x7f03efef87c0>,
 <Reaction r_1589 at 0x7f03efef8dc0>,
 <Reaction r_1598 at 0x7f03efe92760>,
 <Reaction r_1604 at 0x7f03efef8b20>,
 <Reaction r_1621 at 0x7f03efe92dc0>,
 <Reaction r_1625 at 0x7f03efe92b80>,
 <Reaction r_1627 at 0x7f03efef8e20>,
 <Reaction r